
**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

**Edición 2021**

---

# Trabajo práctico entregable - Parte 2

In [ ]:
import io
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn

seaborn.set_context('talk')

## Lectura del dataset

En la notebook 00 se explican los detalles de la siguiente sección.

In [ ]:
url = 'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/sysarmy_survey_2020_processed.csv'
df = pd.read_csv(url)

In [ ]:
df[:3]

,profile_gender,profile_age,work_country,work_province,profile_years_experience,work_years_in_company,work_years_in_current_position,work_people_in_charge_of,profile_studies_level,profile_studies_level_state,profile_career,profile_university,profile_specialization_courses,profile_open_source_contributions,profile_programming_as_hobby,work_role,tools_platform,tools_programming_languages,tools_frameworks,tools_data_bases,tools_qa_testing,tools_IDEs,tools_work_pc_os,tools_cellphone_os,work_on_call_duty,salary_on_call_duty_charge,work_on_call_duty_charge_type,work_contract_type,salary_monthly_BRUTO,salary_monthly_NETO,salary_in_usd,salary_satisfaction,salary_comparison_last_semester,salary_has_bonus,salary_bonus_tied_to,salary_inflation_adjustment_2020,salary_percentage_inflation_adjustment_2020,salary_month_last_inflation_adjustment,work_has_violence_situations,profile_sexual_orientation,profile_has_disabilities,profile_has_disabilities_hiring_difficulties,company_employee_number,company_main_activity,company_recommended,company_diversity_policies,company_extra_benefits,company_best_companies_city
0,Mujer,26,Argentina,Ciudad Autónoma de Buenos Aires,3.0,3.0,3.0,0,Universitario,En curso,Ingeniería en Sistemas de Información,UTN - Universidad Tecnológica Nacional,"Sí, los pagó un empleador",No,Sí,Consultant,Ninguna,Ninguno,Ninguno,Ninguna,Ninguna,Ninguno,GNU/Linux,Android,No,0.0,Bruto,Full-Time,53000.0,43000.0,NaN,1,1,No,No recibo bono,No,0.0,0,En mi trabajo actual,Homosexual,NaN,NaN,501-1000,Servicios / Consultoría de Software / Digital,7,2,"Capacitaciones y/o cursos, Comidas pagas / sub...",NaN
1,Hombre,29,Argentina,Corrientes,5.0,2.0,2.0,4,Universitario,En curso,Ingeniería en Sistemas de Información,UTN - Universidad Tecnológica Nacional,"Sí, de forma particular, Sí, los pagó un emple...",No,Sí,Developer,"Docker, Google Cloud Platform/App Engine","HTML, Javascript, Python","Django, jQuery","Informix, MySQL, PostgreSQL",Ninguna,"IPython / Jupyter, Sublime Text, Visual Studio...",GNU/Linux,Android,No,0.0,Bruto,Full-Time,79000.0,63000.0,NaN,2,2,Menos de un sueldo,Performance de la compañía,Uno,10.0,1,Jamás,Heterosexual,Visual,No,201-500,Otras industrias,8,9,"Horarios flexibles, Stock options / RSUs, Viát...",NaN
2,Mujer,22,Argentina,Ciudad Autónoma de Buenos Aires,2.0,0.0,0.0,0,Secundario,Completado,NaN,NaN,"Sí, de forma particular, Sí, los pagó un emple...",No,Sí,Infosec,"Linux, VMWare",Ninguno,Ninguno,Ninguna,Ninguna,Ninguno,Windows,iOS,No,0.0,Porcentaje de mi sueldo,Full-Time,160000.0,127000.0,NaN,3,3,Un sueldo,Performance individual,No,0.0,0,En un trabajo anterior,Bisexual o queer,NaN,No,2001-5000,Otras industrias,6,9,"Clases de gimnasia online, Comidas pagas / sub...",NaN


In [ ]:
df[['profile_gender', 'salary_monthly_NETO']].groupby('profile_gender').describe()

salary_monthly_NETO                ...                     
                             count          mean  ...       75%        max
profile_gender                                    ...                     
Hombre                      4944.0  98836.063558  ...  110000.0  2080000.0
Mujer                        908.0  73501.414465  ...   92000.0   800000.0
Otros                         31.0  95441.354839  ...  117500.0   380000.0

[3 rows x 8 columns]

In [ ]:
df[df.salary_monthly_NETO > 1000]\
  [['profile_gender', 'salary_monthly_NETO']].groupby('profile_gender').describe()

salary_monthly_NETO                 ...                     
                             count           mean  ...       75%        max
profile_gender                                     ...                     
Hombre                      4815.0  101481.624343  ...  110000.0  2080000.0
Mujer                        863.0   77328.913778  ...   93000.0   800000.0
Otros                         28.0  105660.714286  ...  122750.0   380000.0

[3 rows x 8 columns]

In [ ]:
alpha = 0.05

In [ ]:
is_man = df.profile_gender == 'Hombre'

groupA = df[(df.salary_monthly_NETO > 1000) & is_man].salary_monthly_NETO
groupB = df[(df.salary_monthly_NETO > 1000) & ~is_man].salary_monthly_NETO

## Ejercicio 1: Estimación

**Consigna:**  Calcular una estimación puntual y un intervalo de confianza de nivel (1-alpha) para la resta entre la media del salario Neto para Hombres menos la media del salario Neto para otros géneros(diferencia de las medias entre el grupoA y grupoB). 
¿Cómo se relaciona este intervalo de confianza con el test de hipótesis?

In [ ]:
#Promedio de salario Neto para ambos grupos
promedio_hombres= sum(groupA)/len(groupA)
print("el promedio de salario de los hombres es:" , promedio_hombres)
promedio_otros= sum(groupB)/len(groupB)
print("el promedio de salario de otros generos es:" , promedio_otros)

#Diferencia de salario Neto entre hombres y otros generos
estimacion_puntual = promedio_hombres - promedio_otros
print("Estimador puntual diferencia de medias entre hombres y otros generos:" , round(Estimacion_puntual, 2))

el promedio de salario de los hombres es: 101481.62434267918
el promedio de salario de otros generos es: 78219.25094276095
Estimador puntual diferencia de medias entre hombres y otros generos: 23262.37


In [ ]:
#intervalo de confianza de nivel (1-alpha)
n = len(groupA)
m = len(groupB)

varianza_hombres = groupA.var()
print("la varianza del grupo A es: ", varianza_hombres)
varianza_otros = groupB.var()
print("la varianza del grupo B es: ", varianza_otros)

desvst_hombres = groupA.std()
print("la desviación estandar del grupo A es: ", desvst_hombres)
desvst_otros = groupB.std()
print("la desviación estandar del grupo B es: ", desvst_otros)


la varianza del grupo A es:  10447968511.734694
la varianza del grupo B es:  3193982222.482429
la desviación estandar del grupo A es:  102215.30468444877
la desviación estandar del grupo B es:  56515.32732350074


grados de libertad, $\alpha = 0.05$, $1-\alpha = 0.95$, $\alpha =0.05$, $\alpha/2 = 0.025$

\begin{equation}
t_{\alpha/2} = t_{0.025},
\end{equation}

por lo tanto buscamos un intervalo de confianza para  $\mu_2 -\mu_1$ con un nivel de confianza del 95%.

luego, $t_{0.025}=$



In [ ]:
from math import sqrt
from scipy import stats 

#funcion welch
def grado_libertad(m1,m2,n1, n2, v1, v2):
    df = (v1 / n1+v2 / n2)**2  /  (((v1/n1)**2 / (n1-1))+((v2/n2)**2 / (n2-1)))
    return (df)

df = grado_libertad(promedio_hombres, promedio_otros, n, m, varianza_hombres, varianza_otros)
print ("el valor de grado de libertad viene dado por: ", df)


t= stats.t.ppf(1-alpha/2, grados_lib)
t

el valor de grado de libertad viene dado por:  2148.0505828160885


1.9603799673831133

In [ ]:
from math import sqrt

def pivotes(dif_medias, t, desv1, desv2, n1, n2):
    y =  t * sqrt(desv1**2/n1 + desv2**2/n2)
    linf =  round (dif_medias - y, 2)
    lsup =  round (dif_medias + y, 2) 
    return (pd.DataFrame(np.array([dif_medias,linf,lsup]).reshape(1,-1),
                         columns=['Diferencia de medias','lim superior','lim inferior']))

pivotes (estimacion_puntual, t, desvst_hombres, desvst_otros, n, m)
#print("Intervalo de confianza buscado con t estadistico de scipy: .", limites2)


,Diferencia de medias,lim superior,lim inferior
0,23262.3734,18559.67,27965.08


. ¿Cómo se relaciona este intervalo de confianza con el test de hipótesis?  Nuestro intervalo de confianza está dado por: $[18559.67, 27965.08]$ para una media de   23262.37. 
Este intervalo dado se relaciona con el test de hipotesis en que ambos se pueden sacar diferentes conclusiones. Con el intervalo de confianza obtengo un intervalo que contiene al verdadero valor del del parametro con probabilidad del (1 - $\alpha$)% mientras que del test obtengo evidencia para rechazar o no la hipotesis nula con un cierto nivel de significancia.

La test de hipótesis nos permite tomar una decisión sobre un fenómeno conociendo la probabilidad de que esa decisión sea verdadera. El test de hipótesis no me va a dar verdades absolutas, sino grados de certeza. Así, la hipótesis nula, si no se encuentra evidencia suficiente para rechazarla, es cierta con un porcentaje de probabilidad de que sea incorrecta.
El alfa es el máximo error posible que estoy dispuesto a aceptar, o el máximo error aceptable en mi toma de decisión. Y es, precisamente, el valor con el que armo mi intervalo de confianza.

## Ejercicio 2: Test de hipótesis



### 2.1 Formalización

Describir formalmente los distintos compenentes de un test de hipótesis para comprobar si la distribución de los salarios es distinta entre los grupos A y B.

**Hipótesis Nula**

$H_0= El promedio de salarios de los hombre es igual al promedio del salario de otros generos.

**Estadístico (Pivote)**
  * Identificar el estadístico
La media muestral
  * Escribir qué distribución tiene bajo $H_0$
Distribucion Aproximadamente Normal $\sim N(0,1)$

### 2.2 P-valor

1. Calcule el p-valor y decida si rechazar o no la hipótesis nula.
2. Interprete el resultado.
3. Los dos grupos de nuestra muestra tienen tamaños muy distintos. ¿Esto afecta al tests?

Links útiles:
* [Test de hipótesis usando scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html)
* [Test de Welch](http://daniellakens.blogspot.com/2015/01/always-use-welchs-t-test-instead-of.html)

In [ ]:
import scipy.stats as st

estadistico, p_valor = st.ttest_ind(groupA, groupB, equal_var=False)

print(estadistico, p_valor)


9.697205976350673 8.605197322133209e-22


In [ ]:
import scipy.stats as st
st.ttest_ind(groupA, groupB, axis=0, equal_var=True, nan_policy='propagate')

Ttest_indResult(statistic=6.608570598021937, pvalue=4.232993821653396e-11)

In [ ]:

print('z_obs =', estadistico)
z_alpha = st.norm.ppf(1 - alpha)
print('z_alpha =', z_alpha)
print('Como z_obs > z_alpha, rechazo H_0')

z_obs = 9.697205976350673
z_alpha = 1.6448536269514722
Como z_obs > z_alpha, rechazo H_0


El p- valor cae por fuera del intervalo de confianza por ende se rechaza la hipotesis nula.

El n muestra es diferente y este puee afectar la calidad del test


### [Opcional] 2.3 Potencia del test

Nuestra muestra, ¿era lo suficientemente grande para detectar si existe o no una diferencia entre los grupos?

1. Utilice la función `tt_ind_solve_power` para calcular el tamaño necesario de la muestra para un poder estadístico de 0.8, 0.9 y 0.95, asumiendo una significancia estadística de 0.05.
2. ¿Cómo intepretan el poder estadístico de un test? Dado su conocimiento de dominio sobre los datos, ¿les parece que esta muestra es lo suficientemente grande para ser representativo de la tendencia general? ¿y para utilizarlo en un juicio penal contra una empresa XX por una causa de discriminación?

[Documentación](https://www.statsmodels.org/stable/generated/statsmodels.stats.power.tt_ind_solve_power.html)

NOTA: este análisis debería hacerse ANTES de recolectar los datos.

In [ ]:
from statsmodels.stats.power import tt_ind_solve_power

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
effect_size = (groupA.mean() - groupB.mean()) / groupB.std()
# nobs1=None  - What we want to know
alpha = 0.05
ratio = len(groupB) / len(groupA)

In [ ]:
def tt_ind_solvepower(effect_siz, alph, powe, rati):
  s=tt_ind_solve_power(effect_size=effect_siz, alpha=alph, power=powe, ratio=rati)
  return s
array_tt = [tt_ind_solvepower(effect_size, alpha, 0.8, ratio), tt_ind_solvepower(effect_size, alpha, 0.9, ratio), tt_ind_solvepower(effect_size, alpha, 0.95, ratio)]
 
print("n muestral para un 80% nivel de confianza", [tt_ind_solvepower(effect_size, alpha, 0.8, ratio)])
print("n muestral para un 90% de nivel de confianza" ,  [tt_ind_solvepower(effect_size, alpha, 0.9, ratio)])
print("n muestral para un 95% de nivel de confianza",[tt_ind_solvepower(effect_size, alpha, 0.95, ratio)] )

n muestral para un 80% nivel de confianza [298.3073379956589]
n muestral para un 90% de nivel de confianza [398.79610014029686]
n muestral para un 95% de nivel de confianza [492.8117730249087]


La potencia del test es:  1 menos la probabilidad de cometer un error de tipo 2. Es decir que es la probabilidad de rechazar la hipótesis nula, que las medias de los sueldos son iguales, cuando en realidad son distintos. 

Si, nuestra muestra es representativa de la tendencia general dado que, con un poder estadistico de 0.95, 0.90 y 0.80 los tamaños muestrales deberían ser de aproximadamente 493,399 y 298. Nuestra muestra supera amplicamente estos valores por lo que es representativa (tamaño muestral de más de 5000).

Si podríamos usar este resultado en un juicio penal considerando que la probabilidad de aceptar la H0 siendo falsa es del 5% (alpha = 0,05), es decir  aceptar que los sueldos de ambos grupos son iguales, cuando en realidad son distintos.

## Ejercicio 3: Comunicación y visualización

**Consigna:** Seleccionen un resultado que les parezca relevante a partir de alguno de los ejercicios del entregable. Diseñe e implemente una comunicación en base a este mensaje, en un archivo PDF.

Elija las palabras y visualización más adecuada para que la comunicación sea entendible, efectiva y se dapte a UNA de las siguientes situaciones:

1. Una sección en un artículo de difusión a presentar como parte de una organización sin fines de lucro.
No más de 1 página A4 (o dos si los gráficos son muy grandes).
  1. Ejemplo: Alguna de las secciones [Los ecosistemas de emprendimiento de América Latina y el Caribe frente al COVID-19: Impactos, necesidades y recomendaciones](https://publications.iadb.org/es/los-ecosistemas-de-emprendimiento-de-america-latina-y-el-caribe-frente-al-covid-19-impactos-necesidades-y-recomendaciones), por ejemplo la sección *2.2. Reacciones de los emprendedores*.
  2. Ejemplo: Alguna de las secciones de [The state of gender pay gap in 2021](https://www.payscale.com/data/gender-pay-gap?tk=carousel-ps-rc-job)
  3. Puntos clave: 
    1. Simpleza de los gráficos.
    2. Comunicación en lenguaje simple a personas que no son necesariamente expertos de dominio.
    2. Selección de UNA oración sobre la que se hace énfasis.
    3. No es necesario que mencionen objetivos ni descripciones del conjunto de datos, se supone que eso ya estaría explicado en otras secciones del informe.

2. Una publicación científica o reporte técnico interno. No más de una página A4:
  2. Ejemplo: La sección de resultados de [IZA DP No. 12914: The Impact of a Minimum Wage Change on the Distribution of Wages and Household Income](https://www.iza.org/publications/dp/12914/the-impact-of-a-minimum-wage-change-on-the-distribution-of-wages-and-household-income).
  2. Ejemplo: Alguna de las secciones de [Temporary reduction in daily global CO2 emissions during the COVID-19 forced confinement](https://www.nature.com/articles/s41558-020-0797-x)
  3. Puntos clave:
    3. Nivel de detalle técnico requerido. Es necesario justificar la validez del análisis.
    4. La idea presentada puede ser más compleja. Pueden asumir que la audiencia tiene conocimiento técnico y va a analizar las visualizaciones en detalle. 
    5. Pueden presentar más en detalle las limitaciones del análisis (significancia estadística, etc.)
    2. No es necesario que mencionen objetivos ni descripciones del conjunto de datos, se supone que eso ya estaría explicado en otras secciones del informe.

3. Un tweet (o post de LinkedIn) para la cuenta de su empresa consultora que hace análisis de datos. El objetivo es promocionar un análisis de datos abiertos que van a incluir en su portfolio:
  1. Ejemplo: [Comparación vacunas covid](https://twitter.com/infobeautiful/status/1381577746527236098?s=20)
  2. Ejemplo: [Tweet del BID](https://twitter.com/el_BID/status/1388508583944507396?s=20). Lo valioso de este tweet es que usaron un único número para transmitir un mensaje. Puede ser algo así, o con un gráfico muy simple.
  3. Ejemplo: [Cambio climático](https://twitter.com/UNFCCC/status/1387732156190011394?s=20) Es un muy buen ejemplo, excepto que el gráfico no se lee nada y hay que entrar a la publicación original.
  3. Ejemplo: [¿Cuánto están los programadores en las empresas?](https://www.linkedin.com/posts/denis-rothman-0b034043_tech-career-work-activity-6793861923269054464-gS6y) (No verificamos la veracidad o seriedad de la fuente).
  4. Puntos clave:
    1. Su audiencia no va a mirar la visualización por más de unos segundos, y no tiene conocimiento técnico.
  3. Tienen que incluir además una *breve* descripción de cómo obtuvieron los datos que están presentando, que no entraría en el tweet.
